# Chapter 7: Functions as First-Class Objects

A first-class object is an object that can be created at runtime, assigned to a variable or element of a data structure, passed as an argument to a function, and returned as a result from a function.

Functions as first-class objects are common in functional languages, but they're also prevalent in other languages (like Python)

In [1]:
def factorial(n):
    """returns n!"""
    return 1 if n < 2 else n * factorial(n - 1)

factorial(20), factorial.__doc__, type(factorial)

(2432902008176640000, 'returns n!', function)

In [3]:
# assigning a variable to a function and calling it through there
fact = factorial

# we also pass factorial as an argument to map, returning an iterable
# where each item is the result of calling the first arg (function)
# on elements of the second arg (iterable)
fact(5), map(factorial, range(11)), list(map(factorial, range(11)))

(120,
 [1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800])

In [7]:
# higher-order functions

# these functions take functions as arguments or returns them as results
# one example is map.
# another is sorted via its key argument
fruits = ['strawberry', 'fig', 'apple', 'pear', 'mango', 'banana']

sorted(fruits, key=len) # sorting by length

['fig', 'pear', 'apple', 'mango', 'banana', 'strawberry']

In [6]:
def reverse(word):
    return word[::-1]

sorted(fruits, key=reverse) # sorting by their reversed spelling

['banana', 'apple', 'fig', 'mango', 'pear', 'strawberry']

In [8]:
# functional languages usually offer map, filter, reduce, and apply.

# in Python, apply is unnecessary since you can write fn(*args, **kwargs)

# list comprehensions and generator expressions do map and filter:

list(map(factorial, filter(lambda n: n % 2, range(6))))

[1, 6, 120]

In [9]:
[factorial(n) for n in range(6) if n % 2] # this does the same thing as above
# but is more readable

[1, 6, 120]

In [ ]:
# reduce is now inside of the functools module

# a common use of reduce is to sum an iterable, which can be done with sum()
from functools import reduce
from operator import add

reduce(add, range(100)), sum(range(100))

# other reducing built-ins include all() and any()

(4950, 4950)

In [ ]:
# anonymous functions (lambda keyword)

# the best use of anonymous functions are as arguments to a higher-order function
# like with sorted above

sorted(fruits, key=lambda word: word[::-1])

# apart from that there's not much use.

['banana', 'apple', 'fig', 'mango', 'pear', 'strawberry']

### Lambda Refactoring Recipe

If a piece of code is hard to read because of a lambda, Fredrik Lundh suggests:

Write a comment explaining what it does

Study the comment and think of a name that describes this comment

Make a function with that name using the lambda

Remove the comment

(Wow... enlightening.)

## The Nine Flavors of Callable Objects

All of these things can have the call operator () applied onto them

User-defined functions (def statements, lambda expressions)

Built-in functions implemented in C (len, time.strftime)

Built-in methods implemented in C (dict.get)

Methods (functions defined in the body of a class)

Classes (calling a class is like calling a function since there's no "new" operator)

Class instances (if a class defines \_\_call\_\_, its instances may be invoked as functions)

Generator functions (functions or methods that use yield in the body. these return generator objects)

Native coroutine functions (functions or methods defined with async def)

Async generator functions (the above two combined. returns asynchronous generator (oo spooky))


In [12]:
# user-defined callable types
import random

class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError("picking from empty BingoCage")
        
    def __call__(self):
        return self.pick()
    

In [18]:
bingoo = BingoCage([i for i in range(1, 61)])

nums = []
while True:
    try:
        nums.append(bingoo())
    except LookupError:
        break

In [21]:
nums[0], len(nums), callable(bingoo)

(56, 60, True)

In [ ]:
# parameter handling in Python is very flexible

# for example, * and ** to unpack iterables and mappings

# the following generates HTML elements
def tag(name, *content, class_=None, **attrs):
    """Generate one or more HTML tags"""
    if class_ is not None: # add class_ to attributes (keyword shenanigans?)
        attrs['class'] = class_
    
    # assemble the HTML attributes for this tag
    attr_pairs = (f' {attr}="{value}"' for attr, value
                    in sorted(attrs.items()))
    attr_str = ''.join(attr_pairs)

    # if content exists, then wrap the content in the tag
    if content:
        elements = (f'<{name}{attr_str}>{c}</{name}'
                    for c in content)
        return '\n'.join(elements)
    # otherwise, this is a self-closing tag
    else:
        return f'<{name}{attr_str} />'

In [ ]:
tag('br'), tag('p', 'hello') # passing no content, and passing content

('<br />', '<p>hello</p')

In [ ]:
print(tag('p', 'hello', 'world')) # passing multiple pieces of content (*content)

<p>hello</p
<p>world</p


In [ ]:
tag('p', 'hello', id=33) # passing a kwarg

'<p id="33">hello</p'

In [ ]:
my_tag = {'name': 'img' , 'title': 'Magic Mountain', 'src': 'sunset.jpg', 'class': 'framed'}

tag(**my_tag) # passing several kwargs, including the named param 'name'
# we can use 'class' here since it's a string and does not conflict with
# the reserved keyword

'<img class="framed" src="sunset.jpg" title="Magic Mountain" />'

In [30]:
# specifying keyword-only arguments by naming them after *
def f(a, *, b):
    return a, b

f(1, b=2)

(1, 2)

In [31]:
f(1, 2)

TypeError: f() takes 1 positional argument but 2 were given

In [32]:
# positional-only parameters using "/"
def divmod(a, b, /): # after the "/" you can specify other arguments
    return (a // b, a % b)

divmod(1, 2)

(0, 1)

In [34]:
# packages for functional programming

# operator provides function equivalents for many operators
# this is useful when you want to use an operator as a function
# for example, with factorial
from functools import reduce
from operator import mul

def factorial(n):
    return reduce(mul, range(1, n + 1))

factorial(20)

2432902008176640000

In [35]:
# itemgetter

metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('São Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

from operator import itemgetter
for city in sorted(metro_data, key=itemgetter(1)):
    print(city) 

# itemgetter(1) is equivalent to lambda fields: fields[1]

('São Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


In [36]:
cc_name = itemgetter(1, 0) # returns a tuple with extracted values

for city in metro_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'São Paulo')


In [39]:
# attrgetter extracts object attributes by name

from collections import namedtuple

LatLon = namedtuple('LatLon', 'lat lon')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')

metro_areas = [Metropolis(name, cc, pop, LatLon(lat, lon))
                for name, cc, pop, (lat, lon) in metro_data]

metro_areas[0].coord.lat

35.689722

In [41]:
from operator import attrgetter

name_lat = attrgetter('name', 'coord.lat') # can take nested attributes too

for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

('São Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


In [42]:
# methodcaller creates a function on the fly
from operator import methodcaller

hyphenate = methodcaller('replace', ' ', '-')
hyphenate("The time has come")

'The-time-has-come'

In [ ]:
# freezing arguments with functools.partial

# given a callable, partial produces a new callable with some arguments
# bound to predetermined values

# this is useful to adapt functions that takes one or more arguments
# to an API that requires a callback with fewer arguments

# (?)

from operator import mul
from functools import partial

triple = partial(mul, 3)

triple(7), list(map(triple, range(1, 10))) # mul would not work with map here

(21, [3, 6, 9, 12, 15, 18, 21, 24, 27])

In [ ]:
# unicode normalization from chapter 5
import unicodedata, functools

nfc = functools.partial(unicodedata.normalize, 'NFC')

s1 = 'cafe\u0301'
s2 = 'café'

s1, s2, s1 == s2, nfc(s1) == nfc(s2)

# OHHH. it's freezing arguments for later repeated use

('café', 'café', False, True)

## Chapter Summary

Functions are first-class. You can assign them to variables, pass them to other functions, store them, and access function attributes

Higher-order functions are common in Python. sorted, min, and max are all examples of those. Map, filter, and reduce have been substituted for list comprehensions and built-in reducers like sum

There are lots of different types of callables